### 1. Query to Retrieve Top 5 Cities with Highest Expenditures and Their Percentage Contribution to Total Credit Card Spends:

```sql
WITH TopCitiesExpenditure AS (
    SELECT TOP 5
        City,
        SUM(Amount) AS Citywise_Spent_Amount
    FROM
        credit_card_transactions
    GROUP BY
        City
    ORDER BY
        Citywise_Spent_Amount DESC
),
TotalCreditCardSpends AS (
    SELECT
        SUM(Amount) AS total_amt
    FROM
        credit_card_transactions
)
SELECT
    tc.City,
    tc.Citywise_Spent_Amount,
    100.0 * tc.Citywise_Spent_Amount / tcc.total_amt AS Percentage_contribution
FROM
    TopCitiesExpenditure tc
INNER JOIN
    TotalCreditCardSpends tcc
ON
    1 = 1;
```

### 2. Query to Obtain Highest Spend Month and Amount Spent for Each Card Type:

```sql
WITH MonthlyCardSpends AS (
    SELECT
        DATEPART(year, Date) AS trans_year,
        DATENAME(month, date) AS trans_month,
        Card_Type,
        SUM(amount) AS spent_amt
    FROM
        credit_card_transactions
    GROUP BY
        DATEPART(year, Date),
        DATENAME(month, date),
        Card_Type
),
RankingByCardType AS (
    SELECT
        *,
        DENSE_RANK() OVER (PARTITION BY card_type ORDER BY spent_amt DESC) AS drank
    FROM
        MonthlyCardSpends
)
SELECT
    trans_year,
    trans_month,
    Card_Type,
    spent_amt
FROM
    RankingByCardType
WHERE
    drank = 1;
```

### 3. Query to Display Transaction Details for Each Card Type at Cumulative 1,000,000 Total Spends:

```sql
SELECT
    City,
    Date,
    Card_Type,
    Exp_Type,
    Gender,
    Amount,
    cumulative_sum
FROM (
    SELECT
        *,
        DENSE_RANK() OVER (PARTITION BY card_type ORDER BY k.cumulative_sum) AS drank
    FROM (
        SELECT
            *,
            SUM(Amount) OVER (PARTITION BY card_type ORDER BY Date, Amount) AS cumulative_sum
        FROM
            credit_card_transactions
    ) k
    WHERE
        k.cumulative_sum >= 1000000
) m
WHERE
    m.drank = 1;
```

### 4. Query to Find the City with the Lowest Percentage Spend for the Gold Card Type:

```sql
WITH GoldSpends AS (
    SELECT
        City,
        SUM(Amount) AS spend_amt_ingold_citywise
    FROM
        credit_card_transactions
    WHERE
        Card_Type = 'Gold'
    GROUP BY
        City,
        Card_Type
),
TotalCitySpends AS (
    SELECT
        City,
        SUM(Amount) AS spent_amt_citywise
    FROM
        credit_card_transactions
    GROUP BY
        City
),
PercentageContribution AS (
    SELECT
        c1.City,
        c1.spend_amt_ingold_citywise AS Citywise_Spent_money_ongold,
        c2.spent_amt_citywise AS Citywise_Spent_money,
        100.0 * c1.spend_amt_ingold_citywise / c2.spent_amt_citywise AS perc_contribution
    FROM
        GoldSpends c1
    JOIN
        TotalCitySpends c2
    ON
        c1.City = c2.City
)
SELECT
    TOP 1 *
FROM
    PercentageContribution
ORDER BY
    perc_contribution;

```

### 5. Query to Print 3 Columns: City, Highest Expense Type, Lowest Expense Type:

```sql
WITH ExpenseTypeSpend AS (
    SELECT
        City,
        Exp_Type,
        SUM(Amount) AS spent_amt
    FROM
        credit_card_transactions
    GROUP BY
        City,
        Exp_Type
),
CityExpenseLimits AS (
    SELECT
        city,
        MIN(spent_amt) AS lowest_spent_amount,
        MAX(spent_amt) AS highest_spent_amount
    FROM
        ExpenseTypeSpend
    GROUP BY
        City
)
SELECT
    c1.City,
    MAX(CASE WHEN c2.highest_spent_amount = c1.spent_amt THEN Exp_Type END) AS highest_expense_type,
    MAX(CASE WHEN c2.lowest_spent_amount = c1.spent_amt THEN Exp_Type END) AS lowest_expense_type
FROM
    ExpenseTypeSpend c1
INNER JOIN
    CityExpenseLimits c2
ON
    c1.City = c2.City
GROUP BY
    c1.City
ORDER BY
    c1.City;
```

### 6. Query to Find Percentage Contribution of Spends by Females for Each Expense Type:

```sql
WITH FemaleExpenseContribution AS (
    SELECT  
        Exp_Type,
        SUM(Amount) AS Exp_type_spent_amount
    FROM
        credit_card_transactions
    WHERE
        Gender = 'F'
    GROUP BY
        Exp_Type
),
TotalFemaleSpends AS (
    SELECT
        SUM(Amount) AS total_spent
    FROM
        credit_card_transactions
    WHERE
        Gender = 'F'
)
SELECT
    Exp_Type,
    FORMAT(100.0 * Exp_type_spent_amount / total_spent, 'f2') AS perc_contribution_spent_female
FROM  
    FemaleExpenseContribution
INNER JOIN
    TotalFemaleSpends
ON
    1 = 1;
```

### 7. Query to Identify Card and Expense Type Combination with Highest Month-over-Month Growth in Jan-2014:

```sql
WITH MonthlyGrowth AS (
    SELECT
        Card_Type,
        Exp_Type,
        DATEPART(year, Date) AS Trans_Year,
        DATEPART(month, Date) AS Trans_Month,
        SUM(Amount) AS total_amount
    FROM
        credit_card_transactions
    GROUP BY
        Card_Type,
        Exp_Type,
        DATEPART(year, Date),
        DATEPART(month, Date)
),
PreviousMonthData AS (
    SELECT
        *,
        LAG(total_amount, 1) OVER (PARTITION BY Card_Type, Exp_Type ORDER BY Trans_Year, Trans_Month) AS prev_month_trans_amount
    FROM
        MonthlyGrowth
),
GrowthPercentage AS (
    SELECT
        *,
        100.0 * (total_amount - prev_month_trans_amount) / prev_month_trans_amount AS growth_per_month
    FROM
        PreviousMonthData
    WHERE
        Trans_Year = 2014 AND Trans_Month = 1
)
SELECT
    TOP 1 *
FROM
    GrowthPercentage
ORDER BY
    growth_per_month DESC;
```

### 8. Query to Find City with Highest Total Spend-to-Total Number of Transactions Ratio During Weekends:

```sql
SELECT  
    TOP 1 City,
    SUM(Amount) AS total_spent,
    COUNT(1) AS Count_of_transaction,
    RATIO = SUM(Amount) / COUNT(1)
FROM
    credit_card_transactions
WHERE


    DATEPART(weekday, Date) IN (7, 1)
GROUP BY
    City
ORDER BY
    RATIO DESC;
```

### 9. Query to Determine City that Took Least Number of Days to Reach 500th Transaction After First Transaction:

```sql
WITH CTE1 AS
(
    SELECT
        City,
        COUNT(1) AS count_of_trans,
        MIN(Date) AS MIN_DATE,
        MAX(DATE) AS MAX_DATE
    FROM
        credit_card_transactions
    GROUP BY
        City
    HAVING
        COUNT(1) >= 500
),
CTE2 AS (
    SELECT
        CITY,
        DATE,
        ROW_NUMBER() OVER (PARTITION BY CITY ORDER BY DATE) AS ROW_NM
    FROM
        credit_card_transactions
    WHERE
        City IN (SELECT City FROM CTE1)
),
CTE3 AS (
    SELECT
        C1.CITY,
        C1.MIN_DATE,
        C1.MAX_DATE,
        C1.count_of_trans,
        C2.DATE
    FROM
        CTE1 AS C1
    INNER JOIN
        CTE2 AS C2
    ON
        C1.CITY = C2.CITY
    WHERE
        C2.ROW_NM = 500
)
SELECT
    CITY,
    MIN_DATE AS TRANS_START_DATE,
    DATE AS TRANS_DATE_FOR500TH_TRANS,
    DATEDIFF(DAY, MIN_DATE, DATE) AS DAYS_TO_REACH_500TH_TRANS
FROM
    CTE3
ORDER BY
    DAYS_TO_REACH_500TH_TRANS;
```

### Key Insights:

1. Devote extra marketing resources and promotional campaigns to the top five cities to leverage their robust spending patterns.
2. Strategize targeted promotional offers or campaigns during peak spending months for each card type to incentivize higher expenditures.
3. Examine the factors contributing to low spending in the identified city and explore tailored marketing strategies or partnerships to boost expenditures in that area.
4. Allocate additional staffing or resources to the city exhibiting the highest spend-to-transaction ratio during weekends to maximize opportunities for increased spending.
5. Recognize market potential and contemplate focused marketing endeavors in the city experiencing the fastest transaction growth to attract new customers and foster business expansion.
6. Design specific product or service offerings tailored for females, considering their significant contribution to spending in particular expense categories.